# Bellingcat EDGAR Tool

EDGAR is a database of corporate filings maintained by the SEC. These filings contain a wealth of quantitative and qualitative information on every legal entity that issues non-exempt securities in the United States.

This tool lets you search the EDGAR database and download the results as a spreadsheet. It was initially developed as part of the Bellingcat Tech Fellowship program, we hope it helps you utilise this incredible, free resource.

> <font color='#ffc107'>⚠️Warning:</font> Using both `search_keywords` and only `company_cik` at the same time might not work correctly, we recommend avoiding using both in a search (you can leave `search_keywords` blank and only specify `company_cik`)

In [ ]:
# @title Search EDGAR { display-mode: "form" }

# @markdown ### ⬅️ Click to search the EDGAR filings with the below parameters

search_keywords = "Volcano Safety" # @param {type:"string"}
start_date = '2024-01-01' # @param {type:"date"}
end_date = '2024-12-31' # @param {type:"date"}

TEXT_SEARCH_FILING_CATEGORIES = [
    None,
    "all_except_section_16",
    "all_annual_quarterly_and_current_reports",
    "all_section_16",
    "beneficial_ownership_reports",
    "exempt_offerings",
    "registration_statements",
    "filing_review_correspondence",
    "sec_orders_and_notices",
    "proxy_materials",
    "tender_offers_and_going_private_tx",
    "trust_indentures",
]

filing_type = "all" # @param ["all", "all_except_section_16", "all_annual_quarterly_and_current_reports", "all_section_16", "beneficial_ownership_reports", "exempt_offerings", "registration_statements", "filing_review_correspondence", "sec_orders_and_notices", "proxy_materials", "tender_offers_and_going_private_tx", "trust_indentures"]
company_cik = "" # @param {type:"string"}

# Handle empty arguments
search_keywords = "\"\"" if search_keywords == "" else search_keywords
filing_type = None if filing_type == "all" else filing_type
company_cik = None if company_cik == "" else company_cik

# Load Python dependencies
import pandas as pd
from datetime import datetime
from google.colab import data_table, files
from ipywidgets import widgets
from IPython.display import display
from IPython import get_ipython

data_table.enable_dataframe_formatter()

# Create the output files
output = f"edgar_search_results_{datetime.now().strftime('%d%m%Y_%H%M%S')}.csv"
logfile = f"./edgar_log_{datetime.now().strftime('%d%m%Y_%H%M%S')}.log"

# Enable Widgets
!jupyter nbextension enable --py widgetsnbextension >> {logfile} 2>&1

# Install the EDGAR search tool on the first run
![ ! -f "edgar_tool_installed" ] && echo -n "Loading the EDGAR Tool. This takes a while (around 1-2 minutes) but only happens for your first search)..." && apt-get install firefox >> {logfile} 2>&1 && pip install edgar-tool >> {logfile} 2>&1 && pip install pandas==1.5.3 >> {logfile} 2>&1 && touch edgar_tool_installed && echo "Loaded."

# Run the tool with the query
!echo -n "Searching EDGAR..."
!edgar-tool text_search {search_keywords} --start_date {start_date} --end_date {end_date} --filing_type {filing_type} --entity_id {company_cik} --output {output} --browser firefox --min-wait 0.5 --max-wait 1.5 >> {logfile} 2>&1
exit_code = get_ipython().__dict__['user_ns']['_exit_code']
!echo "Done."

# Error handling
if exit_code != 0:
  print("\x1b[31m Something went wrong with the EDGAR tool. This is commonly because there were no results for your query, check you search and try again.\x1b[0m")
else:
  try:
    # Load results
    results = pd.read_csv(output)

    # Show download button
    btn = widgets.Button(description='Download Results')
    btn.on_click(lambda x: files.download(output))
    display(btn)

    # Display the results in a data table
    display(results)
  except FileNotFoundError as e:
    print("\x1b[31m Something went wrong with the EDGAR tool, please get in touch at contact-tech@bellingcat.com and help us improve the tool for everyone. \x1b[0m")

